In [1]:
import json
import collections
from urllib.request import urlopen
import re
import romkan
import pickle
import os

In [2]:
def init():
    global articles, word_list, list_of_articles, list_of_new_articles
    articles = collections.namedtuple('articles', ['title', 'date', 'url', 'text'])
    word_list = list()
    list_of_articles = list()
    list_of_new_articles = list()

def update():
    global articles, word_list, list_of_articles, list_of_new_articles
    request = urlopen("http://www3.nhk.or.jp/news/easy/news-list.json")
    data = json.loads(request.read().decode('utf-8-sig'))

    if os.path.isfile('word_list.p'):
        word_list = pickle.load(open('word_list.p', 'rb'))
        list_of_articles = pickle.load(open('list_of_articles.p', 'rb'))

    for date in data[0].keys():
        url = 'http://www3.nhk.or.jp/news/easy/'+ data[0][date][0]['news_id'] +'/'+ data[0][date][0]['news_id'] +'.out.json'
        article = articles(data[0][date][0]['title_with_ruby'], date, url,
                           json.loads(urlopen(url).read().decode('utf-8-sig')).get('text', ''))
        if article not in list_of_articles:
            list_of_articles.append(article)
            list_of_new_articles.append(article)

    phrase = ''

    for article in list_of_articles:
        article_json = json.loads(urlopen(article[2]).read().decode('utf-8-sig')) #loads the json for the article
        
        for pronunciation in article_json['morph']:
            furi_list = pronunciation.get('ruby', [dict()])
            for furi_dict in furi_list:
                furigana = furi_dict.get('r', furi_dict.get('s', ''))
                if re.match(u'[\u3040-\u309F]+', furigana) != None:
                    hiragana = re.match(u'[\u3040-\u309F]+', furigana) #hiragana
                    if hiragana != None: #check for hiragana
                        phrase += hiragana.string
                elif phrase != '':
                    word_list.append(phrase)
                    phrase = ''

    pickle.dump(word_list, open("word_list.p", "wb"))
    pickle.dump(list_of_articles, open("list_of_articles.p", "wb"))

In [3]:
init()

In [4]:
update()

In [10]:
list_of_articles

[articles(title='<ruby>北朝鮮<rt>きたちょうせん</rt></ruby>とマレーシア\u3000<ruby>国<rt>くに</rt></ruby>の<ruby>関係<rt>かんけい</rt></ruby>がどんどん<ruby>悪<rt>わる</rt></ruby>くなる', date='2017-03-08', url='http://www3.nhk.or.jp/news/easy/k10010901921000/k10010901921000.out.json', text='北朝鮮とマレーシア\u3000国の関係がどんどん悪くなる 北朝鮮のトップのキム・ジョンウン朝鮮労働党委員長の兄のキム・ジョンナムさんが２月１３日にマレーシアで殺されました。マレーシアの警察は北朝鮮の男などが事件に関係していると考えています。しかし、北朝鮮は、マレーシアの警察は正しく調べていないと言って怒りました。 北朝鮮は７日、北朝鮮にいるマレーシアの人を国から出さないと決めました。マレーシアにいる北朝鮮の人が安全だとわかるまでだと言っています。 マレーシアの首相は「北朝鮮が決めたことは世界の国が決めたルールを全然守っていない」と言って怒りました。そして、マレーシアにいる北朝鮮の人は国から出さないと言いました。 キム・ジョンナムさんが殺されたあと、マレーシアと北朝鮮の関係はどんどん悪くなっています。      '),
 articles(title='トランプ<ruby>大統領<rt>だいとうりょう</rt></ruby>が<ruby>新<rt>あたら</rt></ruby>しい<ruby>命令<rt>めいれい</rt></ruby>「６つの<ruby>国<rt>くに</rt></ruby>の<ruby>人<rt>ひと</rt></ruby>を<ruby>入<rt>い</rt></ruby>れない」', date='2017-03-07', url='http://www3.nhk.or.jp/news/easy/k10010901111000/k10010901111000.out.json', text='トランプ大統領が新しい命令「６つの国の人を入れない」 アメリカのトランプ大統領は６日、中東とアフリカの６つの国の人をアメリカに入れない新しい命令を出しました